<a href="https://colab.research.google.com/github/fjadidi2001/Cyber-Attack-Detection/blob/main/CyberAttackDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FlowChart

In [1]:
from graphviz import Digraph

dot = Digraph(comment='Cyber Attack Detection Flowchart')
dot.node('A', 'Start')
dot.node('B', 'Define Problem & Scope')
dot.node('C', 'Collect Dataset')
dot.node('D', 'Preprocess Data')
dot.node('E', 'Perform EDA')
dot.node('F', 'Select Model')
dot.node('G', 'Train Model')
dot.node('H', 'Evaluate Model')
dot.node('I', 'Optimize Model')
dot.node('J', 'Deploy Model')
dot.node('K', 'Monitor & Retrain')
dot.node('L', 'End')

dot.edges(['AB', 'BC', 'CD', 'DE', 'EF', 'FG', 'GH', 'HJ', 'JK', 'KL'])
dot.edge('H', 'I', label='If performance poor')
dot.edge('I', 'G', label='Retrain')

dot.render('flowchart', format='png', view=True)

'flowchart.png'